<a href="https://colab.research.google.com/github/Xiaoyan-Adele/Political-Motivation-behind-French-EV-Promotion/blob/master/Political-Motivation-behind-French-EV-Promotion/Code/LDA_Topic_Modeling_with_Gensim_Team_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Topic model aims at discover the uncovering structure in a collection of texts. In project, we would experiment with documents released on using **keyword research "véhicules électriques"**. The corpus was scrapped from the the French government website. 

Our topic modeling used the **Lantent Dirichlet Allocation (LDA)** method using the Gensim library in jupyter notebook editor.

The Colab version is for more convenient editing of the code description and results presentation.



## LDA Implementation Outlines


1. Library import
2. Data cleaning
3. Preparing data for LDA analysis
4. Model training and result visualization




### 1. Import most of the necessary libraries to set up the operation environment

In [0]:
# Basics
import re
import string
import numpy as np
import pandas as pd
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

unable to import 'smart_open.gcs', disabling that module


In [0]:
# read the csv file into evarticles 
evarticles = pd.read_csv(r'（hided for privacy concern）\Political-Motivation-behind-French-EV-Promotion\Code\evarticles\evarticle.csv',encoding = 'utf-8')
evarticles_df = pd.DataFrame(evarticles)

#print head
evarticles_df.head(2)

,content
0,\n\n15 janvier 2020 - DiscoursDiscours du Prem...
1,\n\n\n \n \n \n \n ...


### 2. Clean the text data

Here we conducted a series of corpus preprocessing accustomed to the characteristics of our dataset. Besides standard practices such as transforming to lowerase, removing numbers, spliting and deleting punctuations, this project took into consideration that we were to model with a corpus that has excessive phrases such as "\n" and the text was in French. Thus we adjust the stopwords accordingly and switched from the initially chosen sklearn library to gensim, since we did not figure out how to build the word dictionary in French with sklearn. 

In the last line fo this section, we inspected the results of our preprocessing and sort them into a seperate column. By doing this, we can compare the preprocessed text with the original one and get an intuitive assessment of the work.

In [0]:
# Load the regular expression library
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

from spacy.lang.fr.stop_words import STOP_WORDS


# Remove punctuation
evarticles_df['content_processed'] = evarticles_df['content'].map(lambda x: re.sub('[,-:\.!?]', '', x))

#remove and \n
evarticles_df['content_processed'] = evarticles_df['content_processed'].map(lambda x: re.sub('\n','', x))

#remove numbers, excessive white space
space_pattern = '\s+'
evarticles_df['content_processed'] = evarticles_df['content_processed'].map(lambda x: re.sub(space_pattern,' ', x))
evarticles_df['content_processed'] = evarticles_df['content_processed'].map(lambda x: re.sub('[0-9]','', x))

# Convert the titles to lowercase
evarticles_df['content_processed'] = evarticles_df['content_processed'].map(lambda x: x.lower().split())


# Initialize spacy 'fr' model, keeping only tagger component (for efficiency)
nlp = spacy.load('fr_core_news_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


evarticles['content_processed'] = lemmatization(evarticles['content_processed'], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#remove stop words
def remove_stopwords(X_list):
    filtered_list =[]
    for it in X_list:
        temp = []
        for word in it:
            if(word not in STOP_WORDS):
                temp.append(word)
        filtered_list.append(temp)
        temp=[] 
    return filtered_list

evarticles['content_processed'] = remove_stopwords(evarticles['content_processed'])

#results inspection
evarticles.head(10)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adele\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adele\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adele\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,content,content_processed
0,\n\n15 janvier 2020 - DiscoursDiscours du Prem...,"[janvier, discoursdiscours, ministre, issu, sé..."
1,\n\n\n \n \n \n \n ...,"[octobre, co, ambition, échelle, européen, véh..."
2,\n \n \n \n Retour\n \n...,"[transition, écologique, puisje, aider, achete..."
3,\n\n\n \n \n \n \n ...,"[prime, conversion, renforcer, accompagner, au..."
4,\n\n\n \n \n \n \n ...,"[juillet, environnement, état, engager, pollut..."
5,\n\n5 décembre 2018 - CommuniquéGroupe VALE en...,"[décembre, communiquégroupe, val, nouvellecalé..."
6,\n\n\n \n \n \n \n ...,"[juin, faire, pays, leader, mondial, technolog..."
7,\n\n\n \n \n \n \n ...,"[décembre, change, janvier, interdiction, plas..."
8,\n\n\n \n [Grand Défi] Développement du st...,"[grand, défi, développemer, stockage, énergie,..."
9,\n\n\n \n \n \n \n ...,"[voiture, autonome, rapport, développer, filiè..."


### 3. Prepare the text for LDA Modeling

The preparation work included formatting the data into bigrams or trigrams. Then we created a dictionary for our text which delegated a unique id for each word in the document. Without vectorizing text, we won't be able to apply the mathmatics behind the LDA model.   

In [0]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(evarticles['content_processed'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[evarticles['content_processed']], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[evarticles['content_processed'][0]]])

['janvier', 'discoursdiscours', 'ministre', 'issu', 'séminaire', 'prononcé', 'faire', 'foibonjour', 'touscontrairement', 'norme', 'mois', 'contrairement', 'peutêtr', 'attendre', 'aller', 'retraite', 'évidemment', 'question', 'matin', 'séminaire', 'chose', 'semestre', 'venir', 'vie', 'public', 'politique', 'prochain', 'mois', 'exclusivement', 'dédier', 'mise_œuvre', 'système', 'retraitele', 'président', 'décembre', 'occasion', 'vœu', 'redire', 'occasion', 'séminaire', 'mimandat', 'échéance', 'électoral', 'dessiner', 'question', 'ralentir', 'continuer', 'lancée', 'prévaloir', 'début', 'quinquennat', 'transformer', 'pays', 'apporter_réponse', 'al', 'question', 'poser', 'longtempsle', 'travail', 'connaissez', 'ensemble', 'grand', 'objectif', 'fixer', 'président_république', 'acte', 'rappeler', 'discours', 'vœu', 'al', 'lister', 'tête', 'chapitre', 'annoncer', 'date', 'tenirun', 'question', 'retraitesnou', 'annoncer', 'hier', 'envoi', 'lettre', 'rectificatif', 'etat', 'reprendre', 'ensemble

In [0]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [0]:
# Form Bigrams
evarticles_bigrams = make_bigrams(evarticles['content_processed'])


In [0]:
evarticles_preprocessed = evarticles['content_processed']


# Create Dictionary
id2word = corpora.Dictionary(evarticles_lemmatized)

# Create Corpus
texts = evarticles_preprocessed

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 2), (8, 4), (9, 2), (10, 1), (11, 1), (12, 1), (13, 2), (14, 2), (15, 1), (16, 1), (17, 3), (18, 1), (19, 1), (21, 2), (22, 1), (24, 1), (25, 1), (26, 3), (27, 5), (29, 1), (30, 1), (31, 6), (32, 2), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (40, 1), (41, 1), (42, 1), (43, 1), (46, 2), (47, 1), (48, 1), (49, 4), (50, 1), (53, 1), (55, 2), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (66, 1), (68, 1), (69, 1), (70, 3), (71, 1), (72, 1), (74, 2), (75, 1), (77, 1), (78, 2), (79, 1), (81, 1), (82, 1), (83, 2), (85, 1), (87, 1), (89, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1), (111, 1), (112, 1), (113, 1), (114, 3), (115, 1), (116, 1), (118, 1), (119, 1), (120, 1), (121, 1), (122, 1), (123, 1), (124, 1), (125, 1), (126, 2), (127, 2), (128, 5), (130, 1), (131, 1), (133, 1), (135, 1

In [0]:
id2word[5]

'acquis'

### 4. Build the model

We consulted the code written by Selva Prabhakaran on *Topic Modeling with Gensim (Python)* and set the hyperparameters accordingly. Although it was not included in this notebook, during the initial experiment, we optimized the first LDA model wishing to improve the coherence rate. However, it turned out that the intial LDA model was among the best-performed models. Thus, we decided to drop the optimization part when drafting the project report.


**Results visualization**

the interactive chart built with pyLDAvis package gave very good representation of topics learned from the text and their associated keywords.

**Topic coherence score**

the score provided a easy-to-read judgement on the quality of our LDA model. 

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [0]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"air" + 0.006*"vêtement" + 0.004*"pollution" + 0.004*"jeter" + '
  '0.003*"textile" + 0.002*"produit" + 0.002*"respirer" + 0.002*"ameublemer" + '
  '0.002*"distributeur" + 0.002*"intolérable"'),
 (1,
  '0.012*"tour" + 0.004*"sportif" + 0.003*"auto" + 0.002*"cycliste" + '
  '0.002*"emporter" + 0.002*"lecteur" + 0.002*"prince" + 0.002*"autovélo" + '
  '0.001*"immédiatement" + 0.001*"accuser"'),
 (2,
  '0.056*"relatif" + 0.029*"décret" + 0.028*"projet" + 0.017*"article" + '
  '0.015*"loi" + 0.015*"contribution" + 0.013*"social" + 0.012*"code" + '
  '0.011*"travail" + 0.011*"établissement"'),
 (3,
  '0.036*"euro" + 0.020*"pourcent" + 0.013*"prime" + 0.011*"janvier" + '
  '0.011*"aide" + 0.009*"an" + 0.009*"véhicule" + 0.008*"mesure" + '
  '0.008*"hausse" + 0.008*"mois"'),
 (4,
  '0.012*"faire" + 0.006*"al" + 0.006*"aller" + 0.006*"public" + 0.006*"grand" '
  '+ 0.005*"politique" + 0.005*"français" + 0.005*"président" + 0.005*"an" + '
  '0.005*"ici"'),
 (5,
  '0.053*"concurren

In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\adele\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.275508 -0.108641       1        1  44.015747
11    -0.232665 -0.111076       2        1  13.482430
13    -0.245345 -0.193967       3        1  10.732805
3     -0.172787 -0.018919       4        1   8.515842
2     -0.226919  0.236815       5        1   6.364054
7     -0.190016 -0.012905       6        1   5.768841
19    -0.232515  0.239596       7        1   5.376971
14     0.035254 -0.055082       8        1   2.651696
12     0.147740  0.004609       9        1   0.647068
10     0.109270  0.039543      10        1   0.571607
6      0.117492  0.017484      11        1   0.555859
5      0.138343 -0.001075      12        1   0.358652
18     0.138275  0.000621      13        1   0.201304
17     0.128574 -0.006494      14        1   0.198783
8      0.128719 -0.004896      15        1   0.134079
1      0.129026 -0.003617      16        1   0.133795
16     0.128801 -0.003514      17        1   0.131122
0      0.127088 -0.006223      18        1   0.101626
15     0.124871 -0.005702      19        1   0.054409
9      0.122304 -0.006556      20        1   0.003315, topic_info=     Category        Freq        Term       Total  loglift  logprob
307   Default  485.000000    ministre  485.000000  30.0000  30.0000
463   Default  304.000000     relatif  304.000000  29.0000  29.0000
421   Default  388.000000      projet  388.000000  28.0000  28.0000
186   Default  224.000000        euro  224.000000  27.0000  27.0000
282   Default  277.000000         loi  277.000000  26.0000  26.0000
...       ...         ...         ...         ...      ...      ...
307   Topic20    0.000207    ministre  485.622101  -4.3540  -9.2379
179   Topic20    0.000207  entreprise  152.849335  -3.1982  -9.2380
266   Topic20    0.000207     janvier  124.085617  -2.9897  -9.2380
514   Topic20    0.000207      social  141.136810  -3.1186  -9.2381
1408  Topic20    0.000207       heure  102.581558  -2.7996  -9.2382

[1122 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1296      7  0.903476          #
5351      5  0.130349          >
5351      9  0.825546          >
5351     11  0.043450          >
985       1  0.860232   Monsieur
...     ...       ...        ...
1394      7  0.025397  évolution
2928      1  0.997714  évènement
1723      6  0.887684        île
1723      7  0.078325        île
861       4  0.968414          €

[1705 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 12, 14, 4, 3, 8, 20, 15, 13, 11, 7, 6, 19, 18, 9, 2, 17, 1, 16, 10])

In [0]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=evarticles_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)




Coherence Score:  0.42734495369042946
